In [ ]:
# --- Imports ---
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from Corpus import Corpus
from Document import *  # Utiliser RedditDocument pour les phrases
from Search_Engine import SearchEngine

def init():
    # --- Charger le CSV (séparateur tabulation) --- 
    df = pd.read_csv(r"./discours_US.csv", sep='\t')
    df.head()

    # --- Créer le dictionnaire id2doc en découpant chaque discours en phrases ---
    id2doc = {}
    doc_id = 0

    for idx, row in df.iterrows():
        speaker = row['speaker']
        texte = row['text']
        date = row['date']
        url = row.get('link', '')

        # Découper en phrases
        phrases = [p.strip() for p in texte.split('.') if p.strip()]

        for phrase in phrases:
            doc = Document(
                titre=f"Discours {idx} - phrase {doc_id}",
                auteur=speaker,
                date=date,
                url=url,
                texte=phrase,
                source = "discours_US.csv"
            )
            id2doc[doc_id] = doc
            doc_id += 1

    # --- Créer l'objet Corpus ---
    global mon_corpus
    mon_corpus = Corpus("Discours US", df['speaker'].unique().tolist(), id2doc)

    # --- Construire vocabulaire et matrices ---
    mon_corpus.construire_vocab()

    mon_corpus.build_mat_TF()

    mon_corpus.build_mat_TFxIDF()

        # Convertir TF et TFxIDF en CSR pour permettre le slicing
    mon_corpus.mat_TF = mon_corpus.mat_TF.tocsr()
    mon_corpus.mat_TFxIDF = mon_corpus.mat_TFxIDF.tocsr()


    # --- Créer le moteur de recherche ---
    global moteur_search
    moteur_search = SearchEngine(mon_corpus, use_tfidf=True)

# --- Création des widgets ---
titre_label = widgets.Label(value="Moteur de recherche de discours US")
mots_clefs_text = widgets.Text(
    value='',
    placeholder='Entrez vos mots-clés séparés par des espaces',
    description='Mots-clés:',
    layout=widgets.Layout(width='70%')
)
top_k_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=20,
    step=1,
    description='Nombre :',
    continuous_update=False
)
search_button = widgets.Button(description="Rechercher", button_style='success')
output_area = widgets.Output()

# --- Fonction de recherche ---
def lancer_recherche(b):
    
    with output_area:
        
        clear_output()
        print("Recherche en cours veuillez patientez..")
        if not 'mon_corpus' in globals() :
            init()
        mots = mots_clefs_text.value.split()
        k = top_k_slider.value
        if mots:
            df_resultats = moteur_search.search(mots, top_k=k)
            clear_output()
            display(df_resultats)
        else:
            print("Veuillez entrer au moins un mot-clé.")

# --- Lier le bouton à la fonction ---
search_button.on_click(lancer_recherche)

# --- Organiser les widgets et afficher l'interface ---
widgets_box = widgets.VBox([
    titre_label,
    widgets.HBox([mots_clefs_text, top_k_slider]),
    search_button,
    output_area
])
display(widgets_box)

# --- Interface COMPARATIVE : Version C (propre + mise en forme) ---

# Nouveau titre pour la comparaison
titre_compare = widgets.Label(
    value="Analyse comparative dans le corpus (fréquences & importance des mots)",
    layout=widgets.Layout(margin='20px 0 10px 0')
)

# Widgets
mot_unique = widgets.Text(
    value='',
    placeholder='Entrez un mot (un seul)',
    description='Mot :',
    layout=widgets.Layout(width='50%')
)

compare_button = widgets.Button(
    description="Comparer",
    button_style='info',
    layout=widgets.Layout(width='20%')
)

output_compare = widgets.Output()

# Fonction de comparaison
def compare_mot(b):
    with output_compare:
        if not mon_corpus in globals() :
            init()
        clear_output()

        mot = mot_unique.value.strip().lower()
        if not mot:
            print("Veuillez entrer un mot.")
            return

        # Vérifier que le vocab existe et contient le mot
        if mon_corpus.vocab is None:
            mon_corpus.construire_vocab()

        if mot not in mon_corpus.vocab:
            print(f"Le mot '{mot}' n'existe pas dans le vocabulaire du corpus.")
            return

        # id du mot dans le vocabulaire (colonne de la matrice)
        idx = mon_corpus.vocab[mot]["id"]

        # --- Fréquence totale (TF) : somme de la colonne idx dans mat_TF ---
        # mat_TF est sparse CSR (docs x termes)
        col_tf = mon_corpus.mat_TF[:, idx]           # colonne (sparse)
        tf_total = int(col_tf.sum())                 # somme totale des occurrences

        # --- Score TF-IDF : on prend la colonne correspondante ---
        col_tfidf = mon_corpus.mat_TFxIDF[:, idx]    # colonne TFxIDF (sparse)
        # convertir en array 1D
        col_tfidf_arr = col_tfidf.toarray().flatten()
        # moyenne sur tous les documents (on peut aussi faire moyenne sur docs qui ont le mot)
        if len(col_tfidf_arr) > 0:
            tfidf_moy = float(col_tfidf_arr.mean())
            # moyenne conditionnelle (sur docs où mot présent) :
            nonzeros = col_tfidf_arr[col_tfidf_arr > 0]
            tfidf_moy_nonzero = float(nonzeros.mean()) if nonzeros.size > 0 else 0.0
        else:
            tfidf_moy = 0.0
            tfidf_moy_nonzero = 0.0

        # --- Top documents contenant le mot (par TF-IDF décroissant) ---
        # on récupère les indices et scores non nuls rapidement
        # (plus rapide : convertir la colonne en COO)
        coo = col_tfidf.tocoo()
        if coo.nnz > 0:
            doc_indices = coo.row
            doc_scores = coo.data
            # trier par score décroissant
            order = doc_scores.argsort()[::-1]
            top_docs = [(int(doc_indices[i]), float(doc_scores[i])) for i in order]
        else:
            top_docs = []

        # Construire l'affichage HTML propre
        html = f"""
        <div style="border:1px solid #ddd; padding:12px; border-radius:8px; background:#fbfbfb;">
          <h3 style="margin:4px 0 8px 0;">Résultats pour le mot : <b>'{mot}'</b></h3>
          <p><b>Fréquence totale (TF)</b> : {tf_total}</p>
          <p><b>Score TF-IDF moyen (tous docs)</b> : {tfidf_moy:.6f}</p>
          <p><b>Score TF-IDF moyen (docs contenant le mot)</b> : {tfidf_moy_nonzero:.6f}</p>
          <hr style="margin:8px 0;">
        """

        # ajouter top documents (limite 10)
        if top_docs:
            html += "<h4>Top documents (par TF-IDF)</h4><ol>"
            for doc_id, score in top_docs[:10]:
                doc = mon_corpus.id2doc.get(doc_id)
                title = doc.titre if doc is not None else f"Doc {doc_id}"
                snippet = (doc.texte[:160] + '...') if (doc is not None and len(doc.texte) > 160) else (doc.texte if doc is not None else '')
                html += f"<li><b>{title}</b> — score {score:.6f}<br><small>{snippet}</small></li>"
            html += "</ol>"
        else:
            html += "<p><i>Aucun document n'a de score TF-IDF non nul pour ce mot.</i></p>"

        # mini-interprétation automatique
        interpr = "faible"
        if tf_total > 50:
            interpr = "élevée"
        elif tf_total > 10:
            interpr = "moyenne"

        html += f"""
          <hr style="margin:8px 0;">
          <p style="color:#444;"><b>Interprétation :</b> la fréquence globale est <b>{interpr}</b>.</p>
        </div>
        """

        display(widgets.HTML(
            f"""
            <div style="
                border:1px solid #ccc;
                padding:15px;
                border-radius:10px;
                background:#f8f9fa;
                margin-bottom: 15px;
                font-family: Arial;">
                
                <h3 style="margin-top:0;">Résultats pour le mot : <b>'{mot}'</b></h3>

                <p><b>Fréquence totale (TF) :</b> {tf_total}</p>

                <p><b>Score TF-IDF moyen :</b> {tfidf_moy:.5f}</p>

                <hr>

                <p style="color:#555;">
                    Analyse automatique :<br>
                    Le mot <b>{mot}</b> apparaît globalement <b>{ 'souvent' if tf_total > 10 else 'peu' }</b> dans le corpus.
                    Son importance TF-IDF est <b>{ 'élevée' if tfidf_moy > 0.01 else 'faible' }</b>, ce qui indique qu'il est
                    { 'spécifique à certains discours' if tfidf_moy > 0.01 else 'répandu de manière uniforme' }.
                </p>
            </div>
            """
        ))

# Lier bouton → fonction
compare_button.on_click(compare_mot)

# --- Affichage interface comparative ---
box_compare = widgets.VBox([
    titre_compare,
    widgets.HBox([mot_unique, compare_button]),
    output_compare
])

display(box_compare)

# --- Widgets pour l'analyse d'un mot ---
mot_text = widgets.Text(
    value='',
    placeholder='Entrez un mot à analyser',
    description='Mot:',
    layout=widgets.Layout(width='50%')
)
analyse_button = widgets.Button(description="Analyser le mot", button_style='info')
output_mot = widgets.Output()

# --- Fonction d'analyse ---
def analyser_mot(b):
    with output_mot:
        clear_output()
        mot = mot_text.value.strip().lower()
        if not mot:
            print("Veuillez entrer un mot.")
            return
        
        if mot not in mon_corpus.vocab:
            print(f"Le mot '{mot}' n'existe pas dans le corpus.")
            return
        
        idx = mon_corpus.vocab[mot]["id"]
        
        # --- Fréquence totale (TF) ---
        tf_total = 0
        freq_max = 0
        doc_max = None
        for doc_id, doc in mon_corpus.id2doc.items():
            freq_doc = doc.texte.lower().split().count(mot)
            tf_total += freq_doc
            if freq_doc > freq_max:
                freq_max = freq_doc
                doc_max = doc
        

        
        # --- Affichage ---
        print(f"Résultats pour le mot : '{mot}'")

        if doc_max:
            print(f"\nLe discours qui utilise le plus ce mot ({freq_max} occurrences) :")
            print(f"Titre : {doc_max.titre}")
            print(f"Auteur : {doc_max.auteur}")
            print(f"Date : {doc_max.date}")
            print(f"Texte : {doc_max.texte}")
        else:
            print(f"Le mot '{mot}' n'apparaît pas dans le corpus.")

# --- Lier le bouton ---
analyse_button.on_click(analyser_mot)

# --- Afficher l'interface ---
widgets_box_mot = widgets.VBox([
    widgets.Label(value="🔎 Analyse d'un mot dans le corpus"),
    widgets.HBox([mot_text, analyse_button]),
    output_mot
])
display(widgets_box_mot)


